<a href="https://colab.research.google.com/github/ArielDief/Imers-o_Alura_Criar_Embedding_Google_Colab/blob/main/ImersaoAluraCriarEmbeddingGoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai

In [3]:
# Import the Python SDK
import numpy as np
import pandas as pd
import google.generativeai as genai


GOOGLE_API_KEY= "Coloque a Sua KEY aqui"
genai.configure(api_key=GOOGLE_API_KEY)


In [4]:
for m in genai.list_models():
  if "embedContent" in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [6]:
title = "A próxima geração de IA para desenvolvedores e Google WorKspace"
sample_text = ("Titulo: A próxima geração de IA para desenvolvedores e Google WorKspace"
"/n"
"Artigo completo:/n"
"/n"
"Gemini API & Google AI Studio: Uma maneira acessivel de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content= sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.045898475, -0.037337676, -0.02179855, 0.01973034, 0.06994353, -0.019047376, -0.014584161, -0.01609473, 0.061431605, 0.06508335, 0.016267536, 0.020461658, -0.062116165, -0.01825744, -0.009474305, -0.01278012, 0.005831828, -0.01960859, -0.025544409, -0.009485805, -0.0065812725, 0.009632991, -0.011639815, -0.06429595, -0.02367739, 0.0095284125, 0.016029077, -0.027608564, -0.025417272, 0.018194966, -0.056137934, 0.059911184, -0.032539994, 0.006169874, -0.028697861, -0.049611446, -0.023036014, -0.03422512, -0.007760939, 0.005244358, 0.0014488137, -0.07947609, -0.020407055, 0.014730137, -0.01228161, -0.01816157, 0.04049531, 0.036151826, 0.017293286, -0.07212159, 0.03308011, 0.029061522, 0.06960901, -0.032038964, 0.0035720929, -0.016177917, 0.027922224, -0.04735615, 0.019199885, -0.006822474, 0.005135981, 0.021422986, -0.011710331, 0.045769423, 0.014325938, -0.05171642, -0.039116982, -0.0009936786, 0.009947907, 0.049665455, 0.00038265428, 0.0040132976, 0.05981982, -0.05871974

In [7]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Titulo" : "A Importância da Educação",
    "Conteudo" : "A educação é a arma mais poderosa que você pode usar para mudar o mundo."}

DOCUMENT2 = {
    "Titulo" : "O Poder da Mente da Educação",
    "Conteudo" : "A mente é tudo. O que você pensa, você se torna."}

DOCUMENT3 = {
    "Titulo" : "A Busca pela Felicidade",
    "Conteudo" : "A felicidade não é algo pronto. Ela vem das suas próprias ações."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [9]:
df = pd.DataFrame(documents)
df.colums = ["Titulo", "Conteudo"]
df

<ipython-input-9-df713a06068c>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.colums = ["Titulo", "Conteudo"]


,Titulo,Conteudo
0,A Importância da Educação,A educação é a arma mais poderosa que você pod...
1,O Poder da Mente da Educação,"A mente é tudo. O que você pensa, você se torna."
2,A Busca pela Felicidade,A felicidade não é algo pronto. Ela vem das su...


In [10]:
model = "models/embedding-001"

In [12]:
def embed_fn(title, text):
  return genai.embed_content(model= model,
                                 content= text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [17]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row ["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,A Importância da Educação,A educação é a arma mais poderosa que você pod...,"[0.014894009, -0.011953056, -0.002965148, 0.01..."
1,O Poder da Mente da Educação,"A mente é tudo. O que você pensa, você se torna.","[-0.007999488, -0.00030781946, 0.023699775, -0..."
2,A Busca pela Felicidade,A felicidade não é algo pronto. Ela vem das su...,"[0.0024323186, 0.009969189, 0.025265085, 0.023..."


In [24]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta= genai.embed_content(model= model,
                                 content= consulta,

                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [26]:
consulta = "Como buscar a felicidade?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

A felicidade não é algo pronto. Ela vem das suas próprias ações.


In [31]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,


}


In [33]:
prompt = f"Reescreva esse texto de uma forma mais descontraida, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

A felicidade não é um pacote pronto que você compra no mercado. Ela é como uma pizza que você mesmo faz, com os ingredientes que você mais curte.
